In [1]:
%pylab inline
from sys import platform
import pandas as pd
from datetime import datetime

if platform == "win32":
    path = 'C:/Users/olive/'
elif platform == "darwin":
    path = '~/'
    
%run SentimentAnalysis.ipynb
%run GeoFeatures.ipynb

Populating the interactive namespace from numpy and matplotlib


In [2]:
# EXPLORATORY ANALYSIS

from wordcloud import WordCloud

def cloud_gen(text):
    long_string = ','.join(list(text.values))

    wordcloud = WordCloud(background_color="white",
                          max_words=5000,
                          contour_width=3,
                          contour_color='steelblue'
                         )

    return wordcloud.generate(long_string)

In [ ]:
### READS IN RAW REDDIT DATA FROM UNIVERSITY ONEDRIVE
### INTO COMMENT AND SUBMISSION DATAFRAMES

c_df = pd.read_csv(path + 'OneDrive - Cardiff University/Individual Project/raw data/comments.csv', index_col ='id')
s_df = pd.read_csv(path + 'OneDrive - Cardiff University/Individual Project/raw data/submissions.csv', index_col ='id')

In [9]:
### LOADS PROCESSED DATA

c_df = pd.read_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/comments.csv')
s_df = pd.read_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/submissions.csv')

In [ ]:
### ATTEMPT TO GEOCODE LOCATIONS FOR EACH SUBMISSION WHERE POSSIBLE

subs2geocode = s_df.sample(500)
subs2geocode['coords'] = get_captions([*subs2geocode['title'].values])

In [ ]:
body_wordcloud = cloud_gen(c_df['body'])
body_wordcloud.to_image()

In [ ]:
pt_wordcloud = cloud_gen(c_df['processed_text'])
pt_wordcloud.to_image()

In [ ]:
### PROCESSING OF RAW COMMENT AND SUBMISSION DATA

# Create a datetime from the 'created_utc' column of each row
c_df['datetime'] = [datetime.fromtimestamp(i) for i in c_df['created_utc']]
s_df['datetime'] = [datetime.fromtimestamp(i) for i in s_df['created_utc']]

In [ ]:
# Perform sentiment analysis on each comment and return the sentiment and compound score
c_df['sentiment'] , c_df['compound_score'] = sentiment_analysis([*c_df['body']])

In [6]:
# TEXT PREPROCESSING
import re

# REMOVING PUNCTUATION
c_df['processed_text'] = c_df['body'].map(lambda x: re.sub('[,\.!?]', '', x))

# SET ALL TEXT TO LOWERCASE
c_df['processed_text'] = c_df['processed_text'].map(lambda x: x.lower())

c_df['processed_text'].head()

id
fhc1f60    do you know what time each day gmt the figures...
fhaw2m3    worrying part of the uk known cases are all pe...
fhawse0    you’d of thought so could be 100 people carryi...
fhazbjz                         damni’ll go there in 2 weeks
fhawk4d    surely it’s going to hit at all large multi-na...
Name: processed_text, dtype: object

In [7]:
### REFORMAT DATAFRAMES

c_df = c_df[['datetime', 'body', 'processed_text', 'sentiment', 'compound_score', 'submission_id', 'parent_id']]
s_df = s_df[['datetime', 'title', 'selftext']]

In [8]:
### SAVE PROCESSED DATA

c_df.to_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/comments.csv')
s_df.to_pickle(path + 'OneDrive - Cardiff University/Individual Project/processed data/submissions.csv')